Start with a simple constant model:

Hierarchical across country, time

No covariates

Just model p(disaster | country, time), where country and time are fixed effects

Model each disaster independently

We need to choose between three possible distributions for the observed data:

- Poisson
- Negative Binomial
- Zero-Inflated Poisson

Do a test of each of these and report which one is the most appropriate

In [1]:
import sys
sys.path.append("..")

import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import os
from os.path import exists

import pymc as pm
import pytensor
import pytensor.tensor as pt
import matplotlib.pyplot as plt
import arviz as az

config = {
    'figure.figsize':(14,4),
    'figure.constrained_layout.use':True,
    'figure.facecolor':'w',
    'axes.grid':True,
    'grid.linewidth':0.5,
    'grid.linestyle':'--',
    'axes.spines.top':False,
    'axes.spines.bottom':False,
    'axes.spines.left':False,
    'axes.spines.right':False
}

plt.rcParams.update(config)

SEED = sum(list(map(ord, 'climate_bayes')))
rng = np.random.default_rng(SEED)

In [2]:
from laos_gggi.emdat_processing import process_emdat
from laos_gggi.plotting import plot_descriptive, _plot_single_kde, subplots_function, subplots_function_regions
from laos_gggi.const_vars import EM_DAT_COL_DICT, DISASTERS_FOUND, INTENSITY_COLS, EM_DAT_COL_DICT, PROB_COLS, REGIONS
from laos_gggi.summary_stats import descriptive_stats_function

# Data

## Climate Data - need to replace with functions, for sake of time just re-using code for today

In [3]:
CO2_URL = "https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_annmean_mlo.csv"
CO2_FILENAME = "noaa_co2.csv"
OCEAN_HEAT_URL = "https://www.ncei.noaa.gov/data/oceans/woa/DATA_ANALYSIS/3M_HEAT_CONTENT/DATA/basin/3month/ohc_levitus_climdash_seasonal.csv"
# https://www.ncei.noaa.gov/access/global-ocean-heat-content/basin_heat_data_monthly.html
OCEAN_HEAT_FILENAME = "ncei_global_ocean_heat.csv"

In [4]:
def process_ocean_heat(data_path="../data"):
    if not exists(data_path):
        os.makedirs(data_path)

    if not os.path.isfile(os.path.join(data_path, OCEAN_HEAT_FILENAME)):
        df_ocean = pd.read_csv(OCEAN_HEAT_URL, header=0, names=['Date','Temp'])
        df_ocean.Date = pd.to_datetime(df_ocean.Date, format='%Y-%m')
        df_ocean.set_index("Date", inplace=True)
        df_ocean.to_csv(os.path.join(data_path, OCEAN_HEAT_FILENAME))

    else:
        df_ocean = pd.read_csv(
            os.path.join(data_path, OCEAN_HEAT_FILENAME),
            index_col=["Date"],
            parse_dates=True,
        )

    return df_ocean

In [5]:
def process_co2(data_path="../data"):
    if not exists(data_path):
        os.makedirs(data_path)

    if not os.path.isfile(os.path.join(data_path, CO2_FILENAME)):
        df_co2 = pd.read_csv(CO2_URL, skiprows=43)
        df_co2['month'] = 1
        df_co2['day'] = 1
        df_co2['Date'] = pd.to_datetime(df_co2[['year', 'month', 'day']])
        df_co2.set_index("Date", inplace=True)
        df_co2.rename(columns={"mean": "co2"}, inplace=True)
        df_co2 = df_co2[["co2"]]
        df_co2.to_csv(os.path.join(data_path, CO2_FILENAME))

    else:
        df_co2 = pd.read_csv(
            os.path.join(data_path, CO2_FILENAME), index_col=["Date"], parse_dates=True
        )

    return df_co2

In [6]:
df_ocean = process_ocean_heat()
df_co2 = process_co2()

In [7]:
df_ocean = df_ocean.resample('YE').mean()
df_ocean.reset_index(inplace=True)
df_ocean['Date'] = df_ocean['Date'] - pd.offsets.YearBegin()
df_ocean.set_index("Date", inplace=True)
df_ocean = df_ocean.iloc[1:-1] #Dropping first and last column with incomplete data
df_ocean = df_ocean + 152 #add back in baseline that was subtracted from the data

In [8]:
df_ocean = 100*(df_ocean - df_ocean.mean())/df_ocean.std()
df_co2 = 100*(df_co2 - df_co2.mean())/df_co2.std()

In [9]:
df_clim = pd.DataFrame.merge(df_ocean,df_co2,how='inner',left_index=True, right_index=True).dropna()

## EM-DAT data

In [ ]:
data = process_emdat()

df_prob_filtered_adjusted = data["df_prob_filtered_adjusted"]
df_inten_filtered_adjusted = data["df_inten_filtered_adjusted"]

In [ ]:
df_prob_filtered_adjusted['dis_count'] = df_prob_filtered_adjusted.iloc[:, 2:].sum(axis=1)

In [ ]:
df = df_prob_filtered_adjusted.reset_index()

In [ ]:
years = df["Start_Year"].drop_duplicates().sort_values().to_list()[:-1] #current year is not complete
countries = df["ISO"].drop_duplicates().sort_values().to_list()

In [ ]:
df_blank = pd.DataFrame(columns=["dis_count"], index=pd.MultiIndex.from_product([years, countries], names=['Start_Year', 'ISO']))

In [ ]:
df_counts = df_prob_filtered_adjusted[["dis_count"]].sort_values(["Start_Year","ISO"]).reset_index().set_index(["Start_Year","ISO"])

In [ ]:
df_filled = df_blank.combine_first(df_counts).fillna(0)


In [ ]:
df_counts = df_filled.reset_index().pivot(index='Start_Year', columns='ISO').iloc[:-1,:]

In [ ]:
df = df_filled.reset_index()

In [ ]:
df['ISO'] = df['ISO'].astype(CategoricalDtype())
df['Start_Year'] = df['Start_Year'].astype(CategoricalDtype())

In [ ]:
df

In [ ]:
POST_PRED_KWARGS = dict(return_inferencedata=True, extend_inferencedata=True)

In [ ]:
COORDS = {'Country_vals':df['ISO'].cat.categories.values, 
          'Year_vals':df['Start_Year'].cat.categories.values,
          "obs_idx": df.index}

with pm.Model(coords=COORDS) as disaster_model:

    b0 = pm.Normal("Intercept", mu=0, sigma=1)
    b1 = pm.Normal("Country", mu=0, sigma=1, dims="Country_vals")
    b2 = pm.Normal("Year", mu=0, sigma=1, dims="Year_vals")

    country_data = pm.ConstantData('country_data', df['ISO'].cat.codes.to_numpy(), dims="obs_idx")
    year_data = pm.ConstantData('year_data', df['Start_Year'].cat.codes.to_numpy(), dims="obs_idx")

    # not making this a Deterministic because it would take the dims of # of datapoints
    theta = b0 + b1[country_data] + b2[year_data]

    # ## Define Poisson likelihood
    y_hat = pm.Poisson("y_hat", mu=pm.math.exp(theta), observed=df['dis_count'].values)
    
    dis_idata = pm.sample()
    dis_idata = pm.sample_posterior_predictive(dis_idata, 
                                                predictions=True, 
                                                **POST_PRED_KWARGS).assign_coords({"obs_idx":df.index})


In [ ]:
pm.model_to_graphviz(disaster_model)

In [ ]:
az.plot_trace(dis_idata, var_names=['Intercept','Year','Country']);

In [ ]:
dis_idata.predictions['y_hat'].mean(axis=0)